### Installation:
```
module load anaconda3
conda create --name env_teaching
conda activate env_teaching

conda install ipykernel -y -q
python -m ipykernel install --user --name env_teaching

conda install -c conda-forge scanpy python-igraph leidenalg
pip install harmonypy
pip install --user scikit-misc
pip install openpyxl
```

## Introduction to scRNA-seq integration
In this practical, we will illustrate the use of Harmony as a possible alternative to the Seurat integration workflow. Compared to other algorithms, Harmony notably presents the following advantages (Korsunsky et al. 2019, Tran et al. 2020):

  -Possibility to integrate data across several variables (for example, by experimental batch and by condition)
  
  -Significant gain in speed and lower memory requirements for integration of large datasets
    
#### Harmony
Harmony applies a transformation to the principal component (PCs) values, using all available PCs, e.g. as pre-computed within the scanpy workflow. In this space of transformed PCs, Harmony uses k-means clustering to delineate clusters, seeking to define clusters with maximum “diversity”. The diversity of each cluster reflects whether it contains balanced amounts of cells from each of the batches (donor, condition, tissue, technology…) we seek to integrate on, as should be observed in a well-integrated dataset. After defining diverse clusters, Harmony determines how much a cell’s batch identity impacts on its PC coordinates, and applies a correction to “shift” the cell towards the centroid of the cluster it belongs to. Cells are projected again using these corrected PCs, and the process is repeated iteratively until convergence.

#### Integration goals
The following tutorial is designed to give you an overview of the kinds of comparative analyses on complex cell types that are possible using the Harmony integration procedure. Here, we address a few key goals:

  -Compare unitegrated and integrated datasets with UMAP visualisation
  
  -Create an ‘integrated’ data assay for downstream analysis
  
  -Identify cell types that are present in both datasets

In [ ]:
import scanpy as sc
import harmonypy as hm
import pandas as pd
#import anndata as ad
import numpy as np

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
#import seaborn as sns

DPI=300
FONTSIZE=20 #42

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(scanpy = True, dpi=80, transparent=True, vector_friendly = True, dpi_save=DPI) 
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Read in the data

Read in the count matrix into an AnnData object, which holds many slots for annotations and different representations of the data. It also comes with its own HDF5-based file format: .h5ad.

The data is from the paper:

Lee, HO., Hong, Y., Etlioglu, H.E. et al. Lineage-dependent gene expression programs influence the immune landscape of colorectal cancer. Nat Genet 52, 594–603 (2020). https://doi.org/10.1038/s41588-020-0636-z

scRNA-seq was performed on colorectal cancer patient samples. 

In the study 2 datasets were generated, a Korean dataset (SMC) and a Belgian dataset (KUL). For the purpose of this tutorial, only the immune cells are included in the dataset (i.e. cancer cells and stromal cells have been removed).

First download the data from QMplus (session2_SMC_KUL_immune_RAWcounts.h5ad) and upload to a suitable directory on your storage space on apocrita. Next, read in the data (you will need to change the path in the cell below) and begin the analysis. 

In [ ]:
### Read in data
# See anndata-tutorials/getting-started for a more comprehensive introduction to AnnData.

adata = sc.read('/data/home/hww990/Notebooks/Teaching/session2_SMC_KUL_immune_RAWcounts.h5ad')
adata

In [ ]:
### Print max and min counts in dataset
print (np.max(adata.X))
print (np.min(adata.X))

In [ ]:
### The data has already been cleaned and filtered by:
# Only cells with percentage mitochondrial reads < 10% were retained
# Only cells with number of detected genes (n_genes_by_counts) > 300 were retained

### Therefore we can skip the QC filtering step in this example (but it is a very important step normally!)

### Print QC metrics

sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'],
             jitter=0, multi_panel=True, groupby='cell_source')

### Normalise + log transform + HVG

Up to this point the data is only available as a count matrix. Counts are representative of molecules that were captured in the scRNA-seq experiment. As not all mRNA molecules in a cell are captured, there is a variability in the total number of counts detected between cells that results from both the number of molecules that were in the cells, and the sampling. As we cannot assume that all cells contain an equal number of molecules (cell sizes can differ substantially), we have to estimate the number of molecules that were initially in the cells. In fact, we don't estimate the exact number of molecules, but instead estimate cell-specific factors that should be proportional to the true number of molecules. These are called size factors. Normalized expression values are calculated by dividing the measured counts by the size factor for the cell.

The basic preprocessing includes assuming all size factors are equal (library size normalization to counts per million - CPM) and log-transforming the count data.

In [ ]:
# keep raw 
adata.layers["raw"] = adata.X.copy() # preserve counts
    
# normalize + log1p 
sc.pp.normalize_total(adata, target_sum=1e4, inplace=True)
adata.layers["normalised"] = adata.X.copy()
sc.pp.log1p(adata)

adata.layers["log1p"] = adata.X.copy()

In [ ]:
adata.raw = adata # keep normalised log1p data in adata.raw, can access later with adata.raw.to_adata()

In [ ]:
print(adata.X.shape)
print(adata.raw.X.shape)

The count data has been normalized and log-transforme d with an offset of 1. The latter is performed to normalize the data distributions. The offset of 1 ensures that zero counts map to zeros. We keep this data in the '.raw' part of the AnnData object as it will be used to visualize gene expression and perform statistical tests such as computing marker genes for clusters.

Notice that we set the .raw attribute of the AnnData object to the normalized and logarithmized raw gene expression for later use in differential testing and visualizations of gene expression. This simply freezes the state of the AnnData object.

### HVG

We extract highly variable genes (HVGs) to further reduce the dimensionality of the dataset and include only the most informative genes. Genes that vary substantially across the dataset are informative of the underlying biological variation in the data. HVGs are used for clustering, trajectory inference, and dimensionality reduction/visualization, while the full data set is used for computing marker genes, differential testing, cell cycle scoring, and visualizing expression values on the data.

**Expects logarithmized data, except when flavor='seurat_v3' in which count data is expected.**

In [ ]:
### Select highly variable genes for each batch i.e. dataset the cells are from (cell_source)
sc.pp.highly_variable_genes(adata,
                            subset=True, # subset for integration (but full lognorm data in .raw)
                            layer='raw',
                            flavor='seurat_v3',
                            n_top_genes=2000,
                            span=0.3,
                            min_disp=0.5,
                            min_mean=0.0125,
                            max_mean=3,
                            batch_key='cell_source'
                           )

The result of the previous highly-variable-genes detection is stored as an annotation in .var.highly_variable and auto-detected by PCA and hence, sc.pp.neighbors and subsequent manifold/graph tools.

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.

In [ ]:
print(adata.X.shape)
print(adata.raw.X.shape)

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.

Scale each gene to unit variance. Clip values exceeding standard deviation 10.

In [ ]:
var_to_regress = ['n_genes_by_counts','pct_counts_mt']
### Regress out effects from total counts per cell and percentage of mitochondrial genes
sc.pp.regress_out(adata, var_to_regress)

### Scale each gene to unit variance. Clip values exceeding standard deviation 10.
sc.pp.scale(adata, max_value=10)

	
Dimensionality reduction with PCA (Principal Component Analysis)

Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering functions sc.tl.louvain/leiden(). In our experience, often a rough estimate of the number of PCs does fine.

In [ ]:
### calculate PCA
sc.tl.pca(adata, svd_solver='arpack')
### Scatter plot for PCA, but we will not use later on
sc.pl.pca(adata, color=['n_genes_by_counts','pct_counts_mt'], color_map='viridis')
### Estimate number of PCs to use: (rough estimate is often fine)
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

### Computing and embedding the neighborhood graph

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. You might simply use default values here.

Number of neighbours depends on the size of the dataset (usually can get away with using 15 neighbours) 

We use 20 PCs

We suggest embedding the graph in two dimensions using UMAP. It is potentially more faithful to the global connectivity of the manifold than tSNE, i.e., it better preserves trajectories.

In [ ]:
#set n_pcs to use
n_pcs = 20

### Plot data prior to integration (batch correction)

In [ ]:
### compute neighbourhood graph
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=n_pcs, random_state=7)
### embed neighbourhood graph
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['total_counts','n_genes_by_counts','pct_counts_mt','pct_counts_ribo',"doublet_score"], color_map='viridis',
          vmax = [15000,6000,20,50,0.5], ncols=5)
sc.pl.umap(adata, color=["cell_source"], color_map="viridis", palette='Set1')
sc.pl.umap(adata, color=["Patient"], color_map="viridis")
###sc.pl.umap(adata, color=['Annotation_scVI'])

### Integrate (batch correct) the data using Harmony

Harmony applies a transformation to the principal component (PCs) values, using all available PCs, e.g. as pre-computed within the scanpy workflow. In this space of transformed PCs, Harmony uses k-means clustering to delineate clusters, seeking to define clusters with maximum “diversity”. The diversity of each cluster reflects whether it contains balanced amounts of cells from each of the batches (donor, condition, tissue, technology…) we seek to integrate on, as should be observed in a well-integrated dataset. After defining diverse clusters, Harmony determines how much a cell’s batch identity impacts on its PC coordinates, and applies a correction to “shift” the cell towards the centroid of the cluster it belongs to. Cells are projected again using these corrected PCs, and the process is repeated iteratively until convergence.

In this instance we want to batch correct the data, due to potential technical differences between the 2 datasets (SMC and KUL). In the metadata (adata.obs) there is a column called 'cell_source' which records whether each cell is from the SMC and KUL dataset. We then run Harmony, which corrects the PCs. The Harmony corrected PCs are then added back into the adata object, stored within adata.obsm['X_pca_harmony']

In [ ]:
### Run Harmony to integrate data
# Harmony corrects the principal components
cat_var_to_regress = ['cell_source']
ho = hm.run_harmony(adata.obsm['X_pca'], adata.obs, cat_var_to_regress, max_iter_harmony=20)
adata.obsm['X_pca_harmony'] = np.transpose(ho.Z_corr) # add corrected Harmony principal components to 

### Construct neighbour graph and embed umap using Harmony corrected PCs

In [ ]:
### Construct neighbourhood graph using corrected principal components generated with Harmony
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=n_pcs, use_rep='X_pca_harmony', random_state=7)
### embed neighbourhood graph
sc.tl.umap(adata)

In [ ]:
### Plot umaps of quality control metrics
sc.pl.umap(adata, color=['total_counts','n_genes_by_counts','pct_counts_mt','pct_counts_ribo',"doublet_score"], color_map='viridis',
          vmax = [15000,6000,20,50,0.5], ncols=5)
### 
sc.pl.umap(adata, color=["cell_source"], color_map="viridis", palette='Set1')
sc.pl.umap(adata, color=["Patient"], color_map="viridis")
###sc.pl.umap(adata, color=['Annotation_scVI','Annotation_scVI_detailed'])

### Clustering and cell annotation

Annotate the following cell types:
    - B cells (marker genes include: CD79A, BANK1)
    - Plasma cells (marker genes: IGHG3, JCHAIN)
    - T cells (marker genes: CD68, CD14, LYZ)
    - T-NK-ILC (T cells/natural killer cells/innate lymphoid cells) (marker genes include: TRAC, CD3D)

In [ ]:
leiden_res = 0.4
sc.tl.leiden(adata, resolution=leiden_res)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4), gridspec_kw={'wspace':0.5})
sc.pl.umap(adata, color='leiden', ax=ax1, show=False)
sc.pl.umap(adata, color='leiden', ax=ax2, show=False, legend_loc='on data')

In [ ]:
marker_dict = {'B':['CD79A','BANK1'],
               'Plasma':['IGHG3','JCHAIN'],
               'Myeloid':["CD68", "CD14", "LYZ"],
               'T-NK-ILC':['TRAC','CD3D'],
               'Cycling':['MKI67','TOP2A','PCNA'] ### Cycling cells can cluster together - if this was hiding biological information what could we do to avoid this? 
              }

In [ ]:
sc.pl.umap(adata, color=['CD79A','BANK1','IGHG3','JCHAIN','TRAC','CD3D','CD68','LYZ', 'MKI67','TOP2A','PCNA'])

### Finding markers for cluster annotation

Differential expression to get DE genes upregulated per cluster : Let us compute a ranking for the highly differential genes in each cluster. For this, by default, the .raw attribute of AnnData is used in case it has been initialized before.

In [ ]:
### Find marker genes
#del adata.uns['log1p']
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon',pts=True, use_raw=True)
sc.pl.rank_genes_groups(adata, n_genes=15, sharey=False)

pval_thresh = 0.05
log2fc_thresh = 0.25
pct_cutoff = 0.1
cluster_de_genes = dict()
for cluster in sorted(set(adata.obs['leiden'])):
    cluster_de_genes[cluster] = sc.get.rank_genes_groups_df(adata,
                                                            group=cluster, 
                                                            key='rank_genes_groups', 
                                                            pval_cutoff=pval_thresh, 
                                                            log2fc_min=log2fc_thresh, 
                                                            log2fc_max=None).sort_values('logfoldchanges',ascending=False)
    cluster_de_genes[cluster] = cluster_de_genes[cluster][cluster_de_genes[cluster]['pct_nz_group'] > pct_cutoff]

### save results to excel to view later
with pd.ExcelWriter('DE_results_cellType.xlsx') as writer:  
    for cluster in list(cluster_de_genes.keys()):      
        cluster_de_genes[cluster].to_excel(writer, sheet_name='cluster{}'.format(cluster))

In [ ]:
### plot dotplot of marker genes
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="leiden",
    key="rank_genes_groups",
    var_names=marker_dict,
    values_to_plot="logfoldchanges",
    cmap="bwr",
    vmin=-4,
    vmax=4,
    min_logfoldchange=1,
    colorbar_title="log fold change",
    save="marker_dotplot.pdf",
    show=True,
)

### Annotate clusters - note that yours may be different if you've used a different resolution for clustering!

In [ ]:
### annotate cells using marker genes expressed within each cluster
annotation_dict = {'0':'T-NK-ILC',
                   '1':'T-NK-ILC',
                   '2':'Myeloid',
                   '3':'T-NK-ILC',
                   '4':'Myeloid',
                   '5':'B',
                   '6':'Plasma',
                   '7':'T-NK-ILC',
                   '8':'T-NK-ILC',
                   '9':'Cycling Myeloid/T',
                   '10':'Myeloid',
                   '11':'Myeloid'
                  }

adata.obs['Cell_type'] = adata.obs['leiden'].map(annotation_dict).astype('category')
print(adata.X.shape)
sc.pl.umap(adata, color=['leiden','Cell_type'])

### Annotation of subpopulations in T-NK-ILC
Often annotating the major cell types is the first stage of an analysis. Often we are more interested in particular sub-populatons of a cell type. 
Next we subset the T-NK-ILC cells and annotate different sub-populations.

Often it is usual after subsetting a cell type to re-call HVGs and repeat the PCA, integration, UMAP and clustering steps because the analysis is then based on HVGs within a cell type. In this tutorial we won't do this however, but to do it you could run something like the below code before going onto to repeat the analysis:

```
adata_raw = sc.read('session2_SMC_KUL_immune_RAWcounts.h5ad') # read in raw counts
adata = adata_raw[adata.obs_names] # subset raw counts by cell names in adata object of subsetted T-NK-ILC
```


In [ ]:
### adata shape prior to subsetting:
adata.X.shape

In [ ]:
### Subset the adata object using a bool...
adata.obs['Cell_type']=='T-NK-ILC'

In [ ]:
### ... actually do the subsetting
adata = adata[adata.obs['Cell_type']=='T-NK-ILC']
adata

In [ ]:
### adata shape post subsetting:
adata.X.shape

In [ ]:
### Find marker genes for just T-NK-ILC
#del adata.uns['log1p']
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon',pts=True, use_raw=True, key_added='rank_genes_groups_T')
sc.pl.rank_genes_groups(adata, n_genes=15, sharey=False, key='rank_genes_groups_T')

pval_thresh = 0.05
log2fc_thresh = 0.25
pct_cutoff = 0.1
cluster_de_genes = dict()
for cluster in sorted(set(adata.obs['leiden'])):
    cluster_de_genes[cluster] = sc.get.rank_genes_groups_df(adata,
                                                            group=cluster, 
                                                            key='rank_genes_groups_T', 
                                                            pval_cutoff=pval_thresh, 
                                                            log2fc_min=log2fc_thresh, 
                                                            log2fc_max=None).sort_values('logfoldchanges',ascending=False)
    cluster_de_genes[cluster] = cluster_de_genes[cluster][cluster_de_genes[cluster]['pct_nz_group'] > pct_cutoff]

with pd.ExcelWriter('DE_results_TNKILC.xlsx') as writer:  
    for cluster in list(cluster_de_genes.keys()):      
        cluster_de_genes[cluster].to_excel(writer, sheet_name='cluster{}'.format(cluster))

In [ ]:
### Annotate T-NK-ILC subpopulations
marker_dict = {'CD4+ T':['CD4'],
               'CD8+ T':['CD8A','CD8B'],
               'CD4+ Treg':["FOXP3", "CTLA4", "IL2RA"],
               'NK':['KLRC1','FCER1G']
              }

In [ ]:
sc.pl.umap(adata, color=['CD4','CD8A','FOXP3','KLRC1'], vmin=1, vmax='p99', use_raw=True, color_map='plasma_r')

In [ ]:
# delete dendrogram so the dendorgram is recalculated in the next step
del(adata.uns['dendrogram_leiden'])

In [ ]:
### plot dotplot of marker genes
sc.pl.rank_genes_groups_dotplot(
    adata,
    groupby="leiden",
    key="rank_genes_groups_T",
    var_names=marker_dict,
    values_to_plot="logfoldchanges",
    cmap="bwr",
    vmin=-4,
    vmax=4,
    min_logfoldchange=1,
    colorbar_title="log fold change",
    save="marker_dotplot_TNKILC.pdf",
    show=True,
)

In [ ]:
### annotate cells
annotation_dict = {'0':'CD4+ T',
                   '1':'CD8+ T',
                   '3':'CD4+ Treg',
                   '7':'NK cells',
                   '8':'CD4+ T',
                   '9':'Cycling T',
                  }

adata.obs['Cell_type'] = adata.obs['leiden'].map(annotation_dict).astype('category')
print(adata.X.shape)
sc.pl.umap(adata, color=['leiden','Cell_type'])

In [ ]:
# Save adata
adata.write('SMC_KUL_immune_subset_TNKILC.h5ad')